In [37]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr

print(os.getcwd())
# os.chdir('../')
print(os.getcwd())

/Users/bb320/Library/CloudStorage/GoogleDrive-burint@bnmanalytics.com/My Drive/Imperial/01_Projects/TeamofRivals/Analysis/Con2vec-1
/Users/bb320/Library/CloudStorage/GoogleDrive-burint@bnmanalytics.com/My Drive/Imperial/01_Projects/TeamofRivals/Analysis/Con2vec-1


In [5]:
df = pd.read_csv('./Output/super_May22/mm_data_agg.csv')

Generate labels of important turns from Transformer model using attention weights

In [ ]:
# TransformerEncoder with padding masks isn't fully supported. 
# torch.nn.TransformerEncoder tries to use an operator (_nested_tensor_from_mask_left_aligned) that hasn’t been implemented for MPS yet.

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
os.environ["PYTORCH_DISABLE_MPS_FALLBACK"] = "1"  # override fallback misbehavior

torch.backends.mps.is_available = lambda: False  # Completely block MPS from being used



#############################################
# 1. Data Preparation
#############################################
def prepare_data(df, feature_cols, label_map):
    """
    Prepares padded turn sequences, attention masks, and labels from the raw DataFrame.

    Args:
        df (pd.DataFrame): Input dataframe with PairID, Turn, features, and Negotiation_Category.
        feature_cols (list): List of feature column names.
        label_map (dict): Mapping of negotiation categories to integers.

    Returns:
        X_padded (Tensor): [B, T, F] padded feature sequences.
        attention_mask (Tensor): [B, T] mask (1 = real, 0 = padding).
        y_tensor (Tensor): [B] conversation-level labels.
        pair_ids (list): List of PairIDs in the batch order.
    """
    df['NegotiationOutcomeLabel'] = df['Negotiation_Category'].map(label_map)

    grouped = df.groupby('PairID')

    X_list, y_list, pair_ids = [], [], []

    for pair_id, group in grouped:
        group = group.sort_values('Turn')
        features = torch.tensor(group[feature_cols].values, dtype=torch.float32)
        label = torch.tensor([group['NegotiationOutcomeLabel'].iloc[0]], dtype=torch.long)
        X_list.append(features)
        y_list.append(label)
        pair_ids.append(pair_id)

    X_padded = pad_sequence(X_list, batch_first=True)
    attention_mask = torch.zeros(X_padded.shape[:2], dtype=torch.bool)
    for i, seq in enumerate(X_list):
        attention_mask[i, :seq.shape[0]] = 1

    y_tensor = torch.cat(y_list)
    return X_padded, attention_mask, y_tensor, pair_ids

#############################################
# 2. Transformer Model Definition
# This class defines a transformer-based model for classifying conversations.
# Each input is a sequence of turns, where each turn is represented by behavioral features (e.g., facial, vocal, linguistic cues).
# The model uses self-attention to learn which turns are most influential in predicting the outcome (e.g., Constructive, Destructive, etc.).
#############################################
class TurnTransformerClassifier(nn.Module):
    def __init__(self, input_dim, model_dim=64, num_heads=4, num_layers=2, num_classes=4, dropout=0.1):
        super().__init__()
        self.embedding = nn.Linear(input_dim, model_dim)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=model_dim,
            nhead=num_heads,
            dim_feedforward=128,
            dropout=dropout,
            batch_first=True
        )

        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.classifier = nn.Linear(model_dim, num_classes)

    def forward(self, x, mask=None):
        # x: input tensor of shape [batch_size, num_turns, num_features]
        # mask: binary tensor [batch_size, num_turns], where 1 = real turn, 0 = padded
        # This function returns:
        # - logits: predicted class scores for each conversation
        # - x: the internal transformer outputs per turn, used for attention-based interpretation
        x = self.embedding(x)
        transformer_mask = ~mask if mask is not None else None
        x = self.transformer(x, src_key_padding_mask=transformer_mask)

        if mask is not None:
            lengths = mask.sum(dim=1).unsqueeze(1)
            pooled = (x * mask.unsqueeze(-1)).sum(dim=1) / lengths
        else:
            pooled = x.mean(dim=1)

        logits = self.classifier(pooled)
        return logits, x  # Return both logits and transformer output for attention analysis

#############################################
# 3. Utilities
#############################################
def get_label_map():
    return {
        'Constructive': 0,
        'Destructive': 1,
        'Friendly': 2,
        'Apathetic': 3
    }

def get_feature_columns(df):
    # Explicitly define the behavioral feature columns from Acknowledgement to Smile
    start_col = 'Acknowledgement'
    end_col = 'Smile'
    cols = df.columns.tolist()
    start_idx = cols.index(start_col)
    end_idx = cols.index(end_col) + 1  # +1 because slicing is exclusive
    return cols[start_idx:end_idx]

#############################################
# 4. Training Loop
#############################################
def train_model(model, X, mask, y, num_epochs=20, batch_size=8, lr=1e-3):

    device = torch.device("cpu")


    # macOS does not support CUDA (NVIDIA GPUs), but if using a compatible system, MPS (Metal Performance Shaders) can accelerate training
    # This block assumes MPS; skip DataParallel which is CUDA-dependent

    model.to(device)
    dataset = TensorDataset(X, mask, y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model.train()
    for epoch in range(num_epochs):
        total_loss, total_correct = 0.0, 0
        for xb, mb, yb in dataloader:
            xb, mb, yb = xb.to(device), mb.to(device), yb.to(device)
            optimizer.zero_grad()
            logits, _ = model(xb, mb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * xb.size(0)
            preds = torch.argmax(logits, dim=1)
            total_correct += (preds == yb).sum().item()

        avg_loss = total_loss / len(dataloader.dataset)
        accuracy = total_correct / len(dataloader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f} - Accuracy: {accuracy:.4f}")

#############################################
# 5. Attention Extraction and Influence Labeling
# This function helps us identify which turns the transformer considers important.
# It uses the L2 norm (magnitude) of each turn's final transformer output as a proxy for importance.
# We flag the top X% of these high-importance turns as 'influential'.
#############################################
def extract_attention_scores(transformer_output, attention_mask, top_pct=0.5):
    """
    Computes mean L2 norm of transformer outputs per turn and flags top X% as influential.

    Args:
        transformer_output (Tensor): [B, T, D] transformer output vectors
        attention_mask (Tensor): [B, T] mask (1 = valid, 0 = padded)
        top_pct (float): Proportion of top turns to flag as influential

    Returns:
        influence_matrix (Tensor): [B, T] binary matrix (1 = influential)
        scores (Tensor): [B, T] importance scores (normalized per conversation)
    """
    norms = transformer_output.norm(dim=-1)  # [B, T]
    norms = norms * attention_mask  # zero out padded turns

    influence_matrix = torch.zeros_like(norms)
    scores = torch.zeros_like(norms)

    for i in range(norms.shape[0]):
        valid_scores = norms[i][attention_mask[i]]
        if len(valid_scores) == 0:
            continue
        threshold = torch.quantile(valid_scores, 1 - top_pct)
        influence_matrix[i] = (norms[i] >= threshold).int()
        scores[i] = norms[i] / valid_scores.max()

    return influence_matrix, scores

#############################################
# 6. Inference Helper
# This function runs the model in evaluation mode on MPS (Apple GPU) or CPU,
# returning the transformer outputs used for influence analysis.
def run_model_inference(model, X, mask):
    """
    Runs the trained transformer model in evaluation mode and returns transformer outputs.

    Args:
        model (nn.Module): Trained TurnTransformerClassifier
        X (Tensor): Padded input tensor [B, T, F]
        mask (Tensor): Attention mask [B, T] (1 = real, 0 = padding)

    Returns:
        transformer_output (Tensor): Transformer output for each turn [B, T, D]
    """
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    model.to(device)
    X = X.to(device)
    mask = mask.to(device)
    model.eval()
    with torch.no_grad():
        _, transformer_output = model(X, mask)
    return transformer_output

#############################################
# 7. Influence Output DataFrame
# This function creates a new DataFrame by tagging each turn as influential (1) or not (0),
# based on the transformer's internal output norms.
def create_influence_dataframe(df, pair_ids, influence_matrix):
    """
    Adds an 'Influential' column to the original dataframe using attention scores.

    Args:
        df (pd.DataFrame): Original dataframe with PairID and Turn.
        pair_ids (list): List of PairIDs matching the model input batch order.
        influence_matrix (Tensor): [B, T] binary tensor of influential turns.

    Returns:
        pd.DataFrame: Original dataframe with an added 'Influential' column (0 or 1).
    """
    influence_records = []

    for i, pair_id in enumerate(pair_ids):
        convo_df = df[df['PairID'] == pair_id].sort_values('Turn').reset_index(drop=True)
        influence_flags = influence_matrix[i][:len(convo_df)].cpu().numpy()
        convo_df['Influential'] = influence_flags
        influence_records.append(convo_df)

    return pd.concat(influence_records, ignore_index=True)


# NOTE: The behavioral features are the turn-level inputs from your dataset,
# passed in via `feature_cols` during `prepare_data()`. These get embedded
# and processed by the transformer in sequence.


In [85]:

# Identify behavioral feature columns
feature_cols = get_feature_columns(df)

# Define label mapping
label_map = get_label_map()

# Prepare model inputs (turn-level tensors, attention masks, labels)
X_padded, attention_mask, y_tensor, pair_ids = prepare_data(df, feature_cols, label_map)

# Initialize model
model = TurnTransformerClassifier(input_dim=len(feature_cols))

# Train model (can be skipped if model is pre-trained)
train_model(model, X_padded, attention_mask, y_tensor)

# Run inference to get per-turn transformer outputs
transformer_output = run_model_inference(model, X_padded, attention_mask)

# Extract influence scores and flags (e.g., top 20% of turns)
influence_matrix, scores = extract_attention_scores(transformer_output, attention_mask, top_pct=0.2)

# Create updated DataFrame with "Influential" column
df_with_influence = create_influence_dataframe(df, pair_ids, influence_matrix)

# Save updated dataset
df_with_influence.to_csv("./Output/super_May22/conversation_with_influence.csv", index=False)
print("✅ Done. Output saved to conversation_with_influence.csv")




Epoch 1/20 - Loss: 1.4189 - Accuracy: 0.1395
Epoch 2/20 - Loss: 1.4263 - Accuracy: 0.3023
Epoch 3/20 - Loss: 1.3751 - Accuracy: 0.2791
Epoch 4/20 - Loss: 1.3749 - Accuracy: 0.3256
Epoch 5/20 - Loss: 1.3198 - Accuracy: 0.3488
Epoch 6/20 - Loss: 1.3216 - Accuracy: 0.3256
Epoch 7/20 - Loss: 1.3089 - Accuracy: 0.3721
Epoch 8/20 - Loss: 1.2843 - Accuracy: 0.3721
Epoch 9/20 - Loss: 1.2744 - Accuracy: 0.4186
Epoch 10/20 - Loss: 1.2181 - Accuracy: 0.4186
Epoch 11/20 - Loss: 1.2163 - Accuracy: 0.3953
Epoch 12/20 - Loss: 1.1747 - Accuracy: 0.4186
Epoch 13/20 - Loss: 1.2241 - Accuracy: 0.4419
Epoch 14/20 - Loss: 1.3349 - Accuracy: 0.3256
Epoch 15/20 - Loss: 1.1550 - Accuracy: 0.4651
Epoch 16/20 - Loss: 1.2567 - Accuracy: 0.3488
Epoch 17/20 - Loss: 1.1285 - Accuracy: 0.4884
Epoch 18/20 - Loss: 1.0921 - Accuracy: 0.5116
Epoch 19/20 - Loss: 1.0493 - Accuracy: 0.4651
Epoch 20/20 - Loss: 1.1372 - Accuracy: 0.5349
✅ Done. Output saved to conversation_with_influence.csv


In [21]:
df.columns

Index(['Pair_Speaker_turn', 'PairID', 'PersonID', 'Speaker',
       'Speaker_original', 'Turn', 'Word', 'StartTime', 'EndTime',
       'Backchannel', 'Overlap', 'Contested', 'Duration',
       'Negotiation_Category', 'Conflict', 'team_viability', 'Sentiment',
       'word_count', 'Acknowledgement', 'Affirmation', 'Agreement', 'Apology',
       'Ask_Agency', 'By_The_Way', 'Can_You', 'Conjunction_Start', 'Could_You',
       'Disagreement', 'Filler_Pause', 'First_Person_Plural',
       'First_Person_Single', 'For_Me', 'For_You', 'Formal_Title',
       'Give_Agency', 'Goodbye', 'Gratitude', 'Hedges', 'Hello',
       'Impersonal_Pronoun', 'Informal_Title', 'Let_Me_Know', 'Negation',
       'Negative_Emotion', 'Please', 'Positive_Emotion', 'Reasoning',
       'Reassurance', 'Second_Person', 'Subjectivity', 'Swearing',
       'Truth_Intensifier', 'Bare_Command', 'YesNo_Questions', 'WH_Questions',
       'Adverb_Limiter', 'Token_count', 'Pitch', 'Vocal Intensity',
       'Vocal Articulation', 

Correlation analysis

In [86]:

def add_precontested_flags(df, contested_col='Contested'):
    """
    Adds 'PreContested' and 'Other' binary columns to the DataFrame.
    - PreContested: 1 if the turn is immediately before a contested turn (same PairID), else 0
    - Other: 1 if the turn is neither contested nor pre-contested, else 0
    """
    df = df.sort_values(by=['PairID', 'Turn']).reset_index(drop=True)
    df['PreContested'] = 0

    contested_idx = df[df[contested_col] == 1].index

    for idx in contested_idx:
        if idx > 0 and df.loc[idx, 'PairID'] == df.loc[idx - 1, 'PairID']:
            if df.loc[idx - 1, contested_col] == 0:
                df.loc[idx - 1, 'PreContested'] = 1

    df['Other'] = ((df[contested_col] == 0) & (df['PreContested'] == 0)).astype(int)
    return df

def compute_binary_correlation_with_pvalues(df):
    """
    Computes correlation matrix and p-values for binary indicators:
    Contested, PreContested, Other, and Influential.

    Returns:
        corr_matrix (DataFrame): Pearson correlation coefficients.
        pval_matrix (DataFrame): Corresponding p-values.
    """
    cols = ['Contested', 'PreContested', 'Other', 'Influential']
    corr_matrix = pd.DataFrame(index=cols, columns=cols, dtype=float)
    pval_matrix = pd.DataFrame(index=cols, columns=cols, dtype=float)

    for col1 in cols:
        for col2 in cols:
            corr, pval = pearsonr(df[col1], df[col2])
            corr_matrix.loc[col1, col2] = corr
            pval_matrix.loc[col1, col2] = pval

    return corr_matrix, pval_matrix

# Example usage:
# df = add_precontested_flags(df)
# corr_matrix, pval_matrix = compute_binary_correlation_with_pvalues(df)
# print("Correlations:\n", corr_matrix.round(2))
# print("\nP-values:\n", pval_matrix.round(4))


In [87]:
df = add_precontested_flags(df_with_influence)
corr_matrix, pval_matrix = compute_binary_correlation_with_pvalues(df)
print("Correlations:\n", corr_matrix.round(2))
print("\nP-values:\n", pval_matrix.round(4))


Correlations:
               Contested  PreContested  Other  Influential
Contested          1.00         -0.31  -0.60         0.05
PreContested      -0.31          1.00  -0.40         0.07
Other             -0.60         -0.40   1.00        -0.03
Influential        0.05          0.07  -0.03         1.00

P-values:
               Contested  PreContested   Other  Influential
Contested        0.0000           0.0  0.0000       0.0005
PreContested     0.0000           0.0  0.0000       0.0000
Other            0.0000           0.0  0.0000       0.0306
Influential      0.0005           0.0  0.0306       0.0000


In [84]:
# Compute L2 norms again
l2_norms = transformer_output.norm(dim=-1).cpu().numpy()  # [B, T]
attention_mask_np = attention_mask.cpu().numpy()

# Prepare list to collect L2 norms in the same order as df
l2_norm_list = []

# You already have pair_ids in the same order as X_padded
# Loop through each conversation and append valid L2 norms per turn
for i, pair_id in enumerate(pair_ids):
    convo_mask = attention_mask_np[i]
    convo_l2 = l2_norms[i]

    # Only take turns where mask == 1
    valid_l2 = convo_l2[convo_mask == 1]

    # Append to list
    l2_norm_list.extend(valid_l2.tolist())

# Now l2_norm_list should have the same length as df
assert len(l2_norm_list) == len(df), f"Expected {len(df)} L2 norms, got {len(l2_norm_list)}"

# Add to dataframe
df_scores = df.copy()
df_scores['L2_norm'] = l2_norm_list


# Step 2: Determine dynamic threshold
total_contested = df['Contested'].sum()

# Sort scores in descending order
sorted_scores = df_scores['L2_norm'].sort_values(ascending=False)

# Determine threshold
dynamic_threshold = sorted_scores.iloc[int(total_contested) - 1] if total_contested > 0 else sorted_scores.max()


# Step 3: Apply threshold to generate new Influential column
df_scores['Influential_dynamic'] = (df_scores['L2_norm'] >= dynamic_threshold).astype(int)

# Step 4: Run overlap analysis

from sklearn.metrics import precision_score, recall_score, f1_score

# Accuracy
num_correct = (df_scores['Contested'] == df_scores['Influential_dynamic']).sum()
num_incorrect = (df_scores['Contested'] != df_scores['Influential_dynamic']).sum()
total_turns = df_scores.shape[0]
accuracy = num_correct / total_turns

# Force to integer just in case
y_true = df_scores['Contested'].astype(int)
y_pred = df_scores['Influential_dynamic'].astype(int)

# Precision / Recall / F1
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)


# Print results
print(f"\nDynamic threshold based on # of Contested turns = {total_contested}")
print(f"Dynamic threshold value = {dynamic_threshold:.4f}")

print("\nAccuracy between Influential_dynamic and Contested:")
print(f"Correct matches: {num_correct}")
print(f"Incorrect matches: {num_incorrect}")
print(f"Accuracy = {accuracy:.3f} ({accuracy*100:.1f}%)")

print("\nPrecision / Recall / F1:")
print(f"Precision = {precision:.3f}")
print(f"Recall    = {recall:.3f}")
print(f"F1-score  = {f1:.3f}")



Dynamic threshold based on # of Contested turns = 1485.2832906248887
Dynamic threshold value = 7.9735

Accuracy between Influential_dynamic and Contested:
Correct matches: 2552
Incorrect matches: 2188
Accuracy = 0.538 (53.8%)

Precision / Recall / F1:
Precision = 0.364
Recall    = 0.368
F1-score  = 0.366
